In [ ]:
!pip install transformers datasets
!pip install torchvision

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00


In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests
model=ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
processor=ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
url1 = 'http://images.cocodataset.org/val2017/000000039769.jpg'
url2='https://www.shutterstock.com/image-vector/airplane-light-blue-color-pink-600nw-2473561343.jpg'
image1 = Image.open(requests.get(url1, stream=True).raw)
image2 = Image.open(requests.get(url2, stream=True).raw)
inputs = processor(images=image1, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class for Image 1:", model.config.id2label[predicted_class_idx])
inputs = processor(images=image2, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class for Image 2:", model.config.id2label[predicted_class_idx])




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

Predicted class for Image 1: Egyptian cat
Predicted class for Image 2: airliner


In [ ]:
from datasets import load_dataset
dataset=load_dataset('cifar10')
train_dataset=dataset['train']
test_dataset=dataset['test']

README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
def preprocess_data(example):
    inputs = processor(example['img'], return_tensors='pt')


    inputs['pixel_values'] = inputs['pixel_values'].squeeze(0)


    inputs['labels'] = example['label']

    return inputs


In [ ]:
train_dataset=train_dataset.map(preprocess_data,batched=True,batch_size=8,num_proc=4)
test_dataset=test_dataset.map(preprocess_data,batched=True,batch_size=8,num_proc=4)

Map (num_proc=4):   0%|          | 0/50000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]